# 2. Adding foregrounds to the simulated CMB
### Original from Jeff McMahon and Renée Hložek 

The previous notebook simulated a purely CMB realisation of the sky. We now want to add in foregrounds to the CMB map. We will start by reading in the parameters we defined in the previous notebook.

## Code preliminaries

In [ ]:
import os
import sys
import numpy as np

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
from constants import *
from cmb_modules import *

In [ ]:
data = './data/'
out = './output/'

### Just some matplotlib and seaborn parameter tuning

In [ ]:
# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

mpl.rcParams.update({'figure.autolayout': False})

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.8',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

# Colorpalettes, colormaps, etc.
sns.set_palette(palette='rocket')

## Section 1.3: Point Source Map

Point sources in CMB maps arise from a number of astrophysical objects including Active Galactic Nuclei (AGN), Dust Star Forming Galaxies (DSFGs), and a bright tail of lensed DSFGs.  All of theses objects are interesting in their own right. For the purposes of our mock skies we approximate these populations as a combination of a faint distribution of sources with a poisson distribution of brightness and small number of very bright sources with and exponentially falling source count. This approximates the distribution of faint and bright sources seen in real CMB maps. The source parameters were chosen by eye to look about the same as real maps at 150 GHz. Detailed source counts have been published and can be consulted to add more reality into these simulations. Publications also detail the frequency scalings of these sources.  In general DSFGs grow brighter at higher frequencies while AGNs have a spectrum that falls with increasing frequencies. These behaviors must be included in multifrequency analyses and simulations.

In [ ]:
# Paramaters to set up the Poisson point source distribution
number_of_sources  = 5000
amplitude_of_sources = 200
number_of_sources_EX = 50
amplitude_of_sources_EX = 1000

In [ ]:
def poisson_source_component(N_x, N_y, pix_size,
                             number_of_sources, amplitude_of_sources):
    """
    Makes a realization of the naive foreground point source map with Poisson
    distribution.
    
    Parameters:
    -----------
    N_x : int
        Number of pixels in the linear dimension along the X-axis.
    N_y : int
        Number of pixels in the linear dimension along the Y-axis.
    pix_size : float
        Size of a pixel in arcminutes.
    number_of_sources : int
        Number of Poisson distributed point sources on the source map.
    amplitude_of_sources : float
        Amplitude of point sources, which serves as the `lambda` parameter
        for the Poisson-distribution used to choose random points from.

    Returns:
    --------
    PSMap : numpy.ndarray of shape (N_x, N_y)
        The Poisson distributed point sources marked on the map in the form of a 2D matrix.
    """
    PSmap = np.zeros([N_x, N_y])
    # We throw random numbers repeatedly with amplitudes given by a Poisson distribution around the mean amplitude
    for i in range(number_of_sources):
        pix_x = int(N_x*np.random.rand())
        pix_y = int(N_y*np.random.rand()) 
        PSmap[pix_x, pix_y] += np.random.poisson(lam=amplitude_of_sources)

    return PSmap

In [ ]:
def exponential_source_component(N_x, N_y, pix_size,
                                 number_of_sources_EX, amplitude_of_sources_EX):
    """
    Makes a realization of the naive foreground point source map with exponential
    distribution.
    
    Parameters:
    -----------
    N_x : int
        Number of pixels in the linear dimension along the X-axis.
    N_y : int
        Number of pixels in the linear dimension along the Y-axis.
    pix_size : float
        Size of a pixel in arcminutes.
    number_of_sources_EX : int
        Number of exponentially distributed point sources on the source map.
    amplitude_of_sources_EX : float
        Amplitude of point sources, which serves as the scale parameter
        for the exponential distribution

    Returns:
    --------
    PSMap : numpy.ndarray of shape (N_x, N_y)
        The exponentially distributed point sources marked on the map in the form of a 2D matrix.
    """
    PSmap = np.zeros([N_x, N_y])
    # We throw random numbers repeatedly with amplitudes given by an exponential
    # distribution around the mean amplitude
    for i in range(number_of_sources_EX):
        pix_x = int(N_x*np.random.rand()) 
        pix_y = int(N_y*np.random.rand()) 
        PSmap[pix_x,pix_y] += np.random.exponential(scale=amplitude_of_sources_EX)

    return PSmap

In [ ]:
# Make a point source map by generating Poisson distributed background and
# adding the exponentially distributed sources over that
PSmap_p = poisson_source_component(N_x, N_y, pix_size, number_of_sources, amplitude_of_sources)
PSmap_e = exponential_source_component(N_x, N_y, pix_size, number_of_sources_EX, amplitude_of_sources_EX)
PSmap = PSmap_p + PSmap_e

hist, bins = np.histogram(PSmap, bins=50, range=[0.001, PSmap.max()])
width = 1.0 * np.diff(bins).min()
centers = (bins[1:] + bins[:-1]) / 2

In [ ]:
fig, axes = plt.subplots(figsize=(12, 12),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})

axes.set_yscale('log')
axes.bar(centers, hist, width=width,
         color=cm.magma(0.93), ec='black', lw=0.5)

axes.set_xlabel('Source amplitude [$\mu$K]', fontsize=axislabelsize, fontweight='bold', color='white')
axes.set_ylabel('Number or pixels', fontsize=axislabelsize, fontweight='bold', color='white')
axes.tick_params(axis='both', which='major', labelsize=axisticksize, colors='white')

plt.show()

In [ ]:
plot_CMB_map(PSmap, X_width, Y_width,
             cmap=None, c_min=-100, c_max=100,
             save=True, no_axis=False, no_grid=True)

The top plot shows a histogram of brightness of the pixels in our source map. The lower plot shows a map of the point source map we have simulated.

In [ ]:
# Poisson
hist_p, bins_p = np.histogram(PSmap_p, bins=50, range=[0.001, PSmap.max()])
width_p = 1.0 * np.diff(bins_p).min()
centers_p = (bins_p[1:] + bins_p[:-1]) / 2
# Exponential
hist_e, bins_e = np.histogram(PSmap_e, bins=50, range=[0.001, PSmap.max()])
width_e = 1.0 * np.diff(bins_e).min()
centers_e = (bins_e[1:] + bins_e[:-1]) / 2

In [ ]:
nrows = 1
ncols = 2
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*12, nrows*12),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})

axes[0].set_yscale('log')
axes[0].bar(centers_p, hist_p, width=width_p,
            color=cm.magma(0.93), ec='black', lw=0.5)
axes[1].bar(centers_e, hist_e, width=width_e,
            color=cm.magma(0.93), ec='black', lw=0.5)

titles = ['Poisson', 'exponential']
for i in range(ncols):
    axes[i].set_title('Histogram of {0} sources'.format(titles[i]),
                      fontsize=axistitlesize, fontweight='bold', color='white')
    axes[i].set_xlabel('Source amplitude [$\mu$K]', fontsize=axislabelsize, fontweight='bold', color='white')
    axes[i].set_ylabel('Number or pixels', fontsize=axislabelsize, fontweight='bold', color='white')
    axes[i].tick_params(axis='both', which='major', labelsize=axisticksize, colors='white')

plt.show()

We can clearly see, that however Possion sources are more abundant, they only contribute to a very narrow slice of source amplitudes in the inverval $\left[ 0, 500 \right]$. In contrast the exponential sources cover a much wider range in the amplitude domain.

## Section 1.4: Sunyaev–Zeldovich map

Clusters of galaxies imprint a subtle distortion into CMB maps that is most apparent on arcminute scales. While clusters of galaxies are named after the galaxies bound within them, the galaxies represent only a small fraction of the matter contained within a cluster. Roughly 80% of the baryons are not contained within galaxies, but rather exist as a cloud of gas bound within the gravitational potential well created by a dark matter halo that caries the vast majority of the mass of the cluster.  Within this well, the dilute gas becomes ionized and heated to temperatures of millions of Kelvin. Occasionally a CMB photon interacts with one of the hot electrons in this ionized gas. This interaction (inverse Compton scattering) gives the CMB photon a boost in energy. Detailed calculations show that this effect (the Sunyaev–Zeldovich or SZ effect) leads to decrement of power at frequencies below the 'null' at 220 GHz and extra power at higher frequencies. This result is redshift independent. Thus the SZ effect provides a clean way to detect clusters of galaxies and the signal which traces the electron density within the cluster. The SZ signal is a reasonably good tracer of cluster mass.

For these simulations we treat each cluster as having a brightness "beta profile", and fix each cluster to have an identical angular size. We draw the distribution of central temperatures from the exponential distribution to simplify the code and reduce the dependance on external libraries. For more accurate simulations, a range of clusters sizes should be used, a distribution of cluster shapes (with more accurate profiles) should be considered, and the number of clusters as a function of mass and redshift should be chosen to match measurements of the cluster mass function.

In [ ]:
# Paramaters to set up the SZ point sources
number_of_SZ_clusters = 500
mean_amplitude_of_SZ_clusters = 50
SZ_beta = 0.86
SZ_theta_core = 1.0

In [ ]:
def beta_function(N_x, N_y,
                  X_width, Y_width, pix_size,
                  SZ_beta, SZ_theta_core):
    """
    Makes a 2D beta function map to mock the intensity spread of Sunyaev–Zeldovich
    sources. 
    
    Parameters:
    -----------
    N_x : int
        Number of pixels in the linear dimension along the X-axis.
    N_y : int
        Number of pixels in the linear dimension along the Y-axis.
    X_width : float
        Size of the map along the X-axis in degrees.
    Y_width : float
        Size of the map along the Y-axis in degrees.
    pix_size : float
        Size of a pixel in arcminutes.
    SZ_beta : float
        desc
    SZ_theta_core : float
        desc

    Returns:
    --------
    beta : numpy.ndarray of shape (N_x, N_y)
    """
    # Calculate distances to the center of the image on the map
    R = make_coordinates(N_x, N_y,
                         X_width, Y_width,
                         absolute=True)
    
    beta = (1 + (R/SZ_theta_core)**2)**((1 - 3*SZ_beta)/2)

    return(beta)

In [ ]:
def SZ_source_component(N_x, N_y,
                        X_width, Y_width, pix_size,
                        number_of_SZ_clusters, mean_amplitude_of_SZ_clusters,
                        SZ_beta, SZ_theta_core):
    """
    Makes a realization of a naive Sunyaev–Zeldovich effect map.

    Parameters:
    -----------
    N_x : int
        Number of pixels in the linear dimension along the X-axis.
    N_y : int
        Number of pixels in the linear dimension along the Y-axis.
    X_width : float
        Size of the map along the X-axis in degrees.
    Y_width : float
        Size of the map along the Y-axis in degrees.
    pix_size : float
        Size of a pixel in arcminutes.
    number_of_SZ_clusters : int
        Number of the Sunyaev–Zeldovich sources on the map.
    mean_amplitude_of_SZ_clusters : float
        Mean amplitude/size of the Sunyaev–Zeldovich sources on the map.
    SZ_beta : float
        desc
    SZ_theta_core : float
        desc

    Returns:
    --------
    SZmap : numpy.ndarray of shape (N_x, N_y)
        The intensity map of the generated SZ sources with beta
        profiles.
    SZcat : numpy.ndarray of shape (3, number_of_SZ_clusters)
        Catalogue of SZ sources, containing (X, Y, amplitude) in each entry
    """

    # Placeholder for the SZ map
    SZmap = np.zeros([N_x,N_y])
    # Catalogue of SZ sources, X, Y, amplitude
    SZcat = np.zeros([3, number_of_SZ_clusters])
    # Make a distribution of point sources with varying amplitude
    for i in range(number_of_SZ_clusters):
        pix_x = int(N_x*np.random.rand())
        pix_y = int(N_y*np.random.rand())
        pix_amplitude = np.random.exponential(mean_amplitude_of_SZ_clusters)*(-1)
        SZcat[0,i] = pix_x
        SZcat[1,i] = pix_y
        SZcat[2,i] = pix_amplitude
        SZmap[pix_x,pix_y] += pix_amplitude

    # Make a beta function
    beta = beta_function(N_x, N_y, X_width, Y_width, pix_size, SZ_beta, SZ_theta_core)

    # Convolve the beta function with the point source amplitude to get the SZ map
    FT_beta = np.fft.fft2(np.fft.fftshift(beta))
    FT_SZmap = np.fft.fft2(np.fft.fftshift(SZmap))
    SZmap = np.fft.fftshift(np.real(np.fft.ifft2(FT_beta.T*FT_SZmap)))

    return SZmap, SZcat, beta

In [ ]:
SZmap, SZcat, beta = SZ_source_component(N_x, N_y,
                                         X_width, Y_width, pix_size,
                                         number_of_SZ_clusters, mean_amplitude_of_SZ_clusters, SZ_beta, SZ_theta_core)

In [ ]:
hist, bins = np.histogram(SZmap, bins=50, range=[SZmap.min(),-10])
width = 1.0 * np.diff(bins).min()
centers = (bins[1:] + bins[:-1]) / 2

In [ ]:
fig, axes = plt.subplots(figsize=(12,12),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
axes.set_yscale('log')
axes.bar(centers, hist, width=width,
         color=cm.magma(0.93), ec='black', lw=0.5)
axes.set_xlabel('Source amplitude [$\mu$K]', fontsize=axislabelsize, fontweight='bold', color='white')
axes.set_ylabel('Number of pixels', fontsize=axislabelsize, fontweight='bold', color='white')
axes.tick_params(axis='both', which='major', labelsize=axisticksize, colors='white')
plt.show()

In [ ]:
plot_CMB_map(SZmap, X_width, Y_width, c_min=-400, c_max=400,
             save=True, save_filename='SZ_sources.png')

The top plot shows the a histogram of the SZ-decrements from our simulated SZ cluster map. The bottom plot shows our simulated SZ map. This map is at 150 GHz.

### Some notes on reality

In reality the cluster radii vary from cluster to cluster.  For reference the radius depends on redshift and mass. The number of clusters as a function of mass and redshift is called the cluster 'mass function' which is a sensitive cosmological proble.

To enhance the realism of our sims, we could divide the simulated cluster sample into an extremely large radius sample (1 cluster with a 30 arcminute radius, comparable to the Comma cluster; the largest cluster on the sky), a large radius bin (10%) clusters with 5 arcminute radius), a medium bin (30%) with 2 arcminute radius, and a small bin (60% with 0.5 arcminute radius).  

## 1.5 Full Sky Map

The sky map is a combination of the CMB anisotropy, a point source map, and an SZ map.  In an appendix we add the impact of CMB lensing.

In [ ]:
# Read in the input CMB spectra
data = np.genfromtxt(data + 'CAMB_fiducial_cosmo_scalCls.dat',)
ell = data[:,0]
DlTT = data[:,1]
ell_err = data[:,2]
DlTT_err = data[:,3]

In [ ]:
# Create a pure CMB map
CMB_I, ell2d, ClTT2d, FT_2d = make_CMB_I_map(ell, DlTT, N_x=N_x, N_y=N_y,
                                             X_width=X_width, Y_width=Y_width, pix_size=pix_size,
                                             random_seed=42)

# Add them all together to get the sky map at a single freuqency
total_map = CMB_I + PSmap + SZmap

In [ ]:
plot_CMB_map(total_map, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=False, no_axis=False, no_grid=True)

This plot shows our simulated map that includes CMB, point source, and SZ cluster signals.  Note that the sources seem brighter than what we saw in the real observed maps.   This is not a mistake, as will be seen after we fold in the beam (point spread function) of the instrument.